In [1]:
import numpy as np
import os
import sys
import imageio
#import skimage.transform
from pathlib import Path
from llff.poses.colmap_wrapper import run_colmap
import llff.poses.colmap_read_model as read_model

In [2]:
dir = Path('/home/nikola/projects/pxo/data/prepared/nerfstudio/living_room/colmap/sparse/0')

In [3]:
# load_colmap_data(dir)

camerasfile = os.path.join(dir, 'cameras.bin')
camdata = read_model.read_cameras_binary(camerasfile)

imagesfile = os.path.join(dir, 'images.bin')
imdata = read_model.read_images_binary(imagesfile)

points3dfile = os.path.join(dir, 'points3D.bin')
pts3d = read_model.read_points3d_binary(points3dfile)

list_of_keys = list(camdata.keys())
cam = camdata[list_of_keys[0]]

h, w, f = cam.height, cam.width, cam.params[0]
hwf = np.array([h,w,f]).reshape([3,1])

w2c_mats = []
bottom = np.array([0,0,0,1.]).reshape([1,4])

names = [imdata[k].name for k in imdata]
print( 'Images #', len(names))
perm = np.argsort(names)
for k in imdata:
    im = imdata[k]
    R = im.qvec2rotmat()
    t = im.tvec.reshape([3,1])
    m = np.concatenate([np.concatenate([R, t], 1), bottom], 0)
    w2c_mats.append(m)

w2c_mats = np.stack(w2c_mats, 0)
c2w_mats = np.linalg.inv(w2c_mats)

poses = c2w_mats[:, :3, :4].transpose([1,2,0])
poses = np.concatenate([poses, np.tile(hwf[..., np.newaxis], [1,1,poses.shape[-1]])], 1)
# must switch to [-u, r, -t] from [r, -u, t], NOT [r, u, -t]
poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], -poses[:, 2:3, :], poses[:, 3:4, :], poses[:, 4:5, :]], 1)    


Images # 606


In [4]:
# initializes an empty list to store 3d points world points position
pts_pos_world = []

for k in pts3d:
    # for each point in the list of 3d points, append it to the list of points
    pts_pos_world.append(pts3d[k].xyz)

# convert list of points to numpy array
pts_pos_world = np.array(pts_pos_world)
#print(f'pts_pos_world.shape: {pts_pos_world.shape}')

# add new axis to pts_pos_world in the middle to get (numpoints, 1, xyz) and transpose 
# to (xyz, numpoints, 1)
pts_pos_world = pts_pos_world[:, np.newaxis, :].transpose([2,0,1])
print(f'pts_pos_world.shape: {pts_pos_world.shape}')
print(f'pts_pos_world[:, 0, :]: \n{pts_pos_world[:, 0, :]}')

print(f'poses.shape: {poses.shape}')
with np.printoptions(suppress=True):
    print(f'poses[:, :, 0]: \n{poses[:, :, 0]}')

# get world camera position from poses
cam_pos_world = poses[:3, 3:4, :]
print(f'cam_pos_world.shape: {cam_pos_world.shape}')
print(f'cam_pos_world[:, :, 0]]: \n{cam_pos_world[:, :, 0]}')

# get last column of 3x3 rotation matrix from poses. 
# the last column of the rotation matrix represents the coordinates of the z-axis of the
# world coordinate system in terms of the camera's local coordinate system
cam_rot_z_world = poses[:3, 2:3, :]
print(f'cam_rot_z_world.shape: {cam_rot_z_world.shape}')
print(f'cam_rot_z_world[:, :, 0]: \n{cam_rot_z_world[:, :, 0]}')

# get displacement vector by subtracting camera world position from the point world position.
# pts_pos_world shape is (3, numpoints, 1) and cam_pos_world shape is (3, 1, numimages) 
# np will perform broadcasting to get (3, numpoints, numimages) for pts_pos_world and
# (3, numpoints, numimages) for cam_pos_world 
displacemet = pts_pos_world - cam_pos_world
print(f'displacemet.shape: {displacemet.shape}')

# myltiply the displacement vector by camera rotation z world coordinate scale the displacement 
# vector by camera rotation z world coordinate?
z_vals = -displacemet * cam_rot_z_world
print(f'z_vals.shape: {z_vals.shape}')

z_vals = sum(z_vals, 0)
print(f'z_vals.shape: {z_vals.shape}')


pts_pos_world.shape: (3, 34856, 1)
pts_pos_world[:, 0, :]: 
[[-4.26072841]
 [-2.11201705]
 [ 5.30605664]]
poses.shape: (3, 5, 606)
poses[:, :, 0]: 
[[  -0.10608506    0.87490473   -0.47253325   -1.21947847 1080.        ]
 [   0.99305713    0.11751033   -0.00537144   -0.19859704 1920.        ]
 [   0.05082803   -0.46982235   -0.88129647    0.07956947  876.07432824]]
cam_pos_world.shape: (3, 1, 606)
cam_pos_world[:, :, 0]]: 
[[-1.21947847]
 [-0.19859704]
 [ 0.07956947]]
cam_rot_z_world.shape: (3, 1, 606)
cam_rot_z_world[:, :, 0]: 
[[-0.47253325]
 [-0.00537144]
 [-0.88129647]]
displacemet.shape: (3, 34856, 606)
z_vals.shape: (3, 34856, 606)
z_vals.shape: (34856, 606)


In [6]:
pts_arr = []
vis_arr = []

for k in pts3d:
    pts_arr.append(pts3d[k].xyz)
    print(f'{k}.xyz: {pts3d[k].xyz}')
    cams = [0] * poses.shape[-1]
    print(f'{k}.cams: {cams}')
    for ind in pts3d[k].image_ids:
        print(f'image_id: {ind}')
        if ind - 1 < len(cams):
            cams[ind-1] = 1
        else:    
            print('ERROR: the correct camera poses for current points cannot be accessed')
            continue
    vis_arr.append(cams)


27995.xyz: [-4.26072841 -2.11201705  5.30605664]
27995.cams: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
print(pts_arr)
print(vis_arr)

[array([-4.26072841, -2.11201705,  5.30605664]), array([-2.3551385 , -1.5179322 ,  4.51019054]), array([-4.28878311, -2.19341806,  5.63542572]), array([-6.12601139, -1.84959325,  5.7631935 ]), array([-2.42372275, -0.54118839,  3.7490976 ])]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0